# Advanced machine learning techniques

From previous lectures you have already learned about different classification algorithms, and also learned how to correctly validate and evaluate the quality of the model. But what if you have already found the best model and improve the accuracy of the model can no longer? In this case, you need to apply more advanced techniques of machine learning, which can be combined with the word "ensembles". An ensemble is a certain aggregate, parts of which form a single whole. From everyday life you know musical ensembles, where several musical instruments are united, architectural ensembles with different buildings, etc.

## Ensembles

A good example of ensembles is Condorcet's theorem "on the jury of juries" (1784). If each member of the jury has an independent opinion, and if the probability of a correct decision of a jury member is greater than 0.5, then the probability of a correct jury decision as a whole increases with the number of jury members and tends to one. If, however, the probability of being right for each member of the jury is less than 0.5, then the probability of taking the correct decision by the jury as a whole decreases monotonically and tends to zero with an increase in the number of jurors. <br>
$ \large N $ - number of jurors<br>
$ \large p $ - probability of correct jury decision<br>
$ \large \mu $ - probability of correct decision of the whole jury<br>
$ \large m $ - $\large [\frac{N+1}{2}]$<br>
$$ \large \mu = \sum_{i = m}^{N} C_N^ip^i (1-p)^{N-i} $$ 
If $ \large p > 0.5 $, then $ \large \mu > p $
If $ \large N \rightarrow \infty $, then $ \large \mu \rightarrow 1 $

Let's look at another example of ensembles - "The Wisdom of the Crowd". Frances Galton visited the market in 1906, where a certain lottery for peasants was held.
They gathered about 800 people, and they tried to guess the weight of the bull that stood in front of them. The bull weighed 1198 pounds. No peasant guessed the exact weight of the bull, but if we calculate the average of their predictions, we will get 1,177 pounds.
This idea of reducing the error was also applied in machine learning.

## Bagging

Suppose there is a training sample $ \large X $. Using bootstrap, we generate from it a sample $ \large X_1, \dots, X_M $. Now on each sample we will train our classifier $ \large a_i (x) $. The resulting classifier will average the answers of all these algorithms (in the case of classification, this corresponds to voting): $$ \large a(x) = \frac{1}{M}\sum_{i = 1}^M a_i(x) $$ This scheme can be represented by the picture below.

![Image](https://github.com/Yorko/mlcourse_open/blob/master/img/bagging.png?raw=true)

### Out-of-bag error

Let $ \large \ell $ objects in the sample. At each step, all objects fall into the sub-sample with a return equally probable, a single object with probability $ \large \frac{1}{\ell}.$ The probability that the object will NOT fall into the subsample (it was not $ \large \ell $ times): $ \large (1 - \frac{1}{\ell})^\ell$. For $ \large \ell \rightarrow + \infty $ we obtain the limit $ \large \frac{1}{e} $. Then the probability of a specific object falling into the subsample $ \large \approx 1 - \frac{1}{e} \approx 63\% $.

![image](https://github.com/Yorko/mlcourse_open/blob/master/img/oob.png?raw=true)

The figure shows an estimate of the oob error. The upper figure is our initial sample, we divide it into a training sample (on the left) and a test sample (on the right). In the figure on the right, we have a grid of squares that perfectly divides our sample. Now we need to estimate the proportion of correct answers in our test sample. The figure shows that our classifier was mistaken in 4 observations, which we did not use for training. Hence, the fraction of the correct answers of our classifier: $ \large \frac{11}{15}*100\% = 73.33\% $.

In [92]:
import pandas as pd
import numpy as np
from scipy import sparse

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

df_train = pd.read_csv('train.csv', 
                       dtype={
                           'question1': np.str,
                           'question2': np.str
                       })
df_train = df_train[~df_train.question2.isnull()]

In [2]:
cv = CountVectorizer(ngram_range=(1,2), max_features=10000)
X_train, X_val, y_train, y_val = train_test_split(df_train[["question1", "question2"]], df_train.is_duplicate, 
                                                  stratify=df_train.is_duplicate, random_state=42)
questions_train = list(np.concatenate([X_train.question1, X_train.question2]))
questions_val = list(np.concatenate([X_val.question1, X_val.question2]))

In [3]:
questions_train_transformed = cv.fit_transform(questions_train)
questions_val_transformed = cv.transform(questions_val)

In [8]:
threshold_train = int(questions_train_transformed.shape[0] / 2)
threshold_val = int(questions_val_transformed.shape[0] / 2)
X_train_new = sparse.hstack((questions_train_transformed[:threshold_train], 
                             questions_train_transformed[threshold_train:]))
X_val_new = sparse.hstack((questions_val_transformed[:threshold_val], 
                           questions_val_transformed[threshold_val:]))

In [19]:
dtc = DecisionTreeClassifier(max_depth=20, class_weight='balanced')
dtc.fit(X_train_new, y_train)
print(log_loss(y_val, dtc.predict_proba(X_val_new)))

0.901928040725


In [11]:
from sklearn.ensemble import BaggingClassifier

In [18]:
bg = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=20, class_weight='balanced'), n_estimators=10, 
                       max_samples=0.7, max_features=0.7, n_jobs=-1, random_state=42)
bg.fit(X_train_new, y_train)
print(log_loss(y_val, bg.predict_proba(X_val_new)))

0.5789404732


We improved our accuracy only using BaggingClassifier with almost default parameters.

## Random Forest

Random Forest is an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees (decision trees that are grown very deep tend to learn highly irregular patterns: they overfit their training sets, i.e. have low bias, but very high variance. Random forests are a way of averaging multiple deep decision trees, trained on different parts of the same training set, with the goal of reducing the variance).
Random decision forests correct for decision trees' habit of overfitting to their training set. <br> 

So we can say that Random Forest is like bootstrapping algorithm with Decision tree (CART) model. Random forests differ in only one way from this general scheme: they use a modified tree learning algorithm that selects, at each candidate split in the learning process, a random subset of the features. This process is called random subspace method or sometimes "feature bagging", and it attempts to reduce the correlation between estimators (decision trees) in an ensemble by training them on random samples of features instead of the entire feature set. <br> 

Main parameters:
* n_estimators — the number of trees in the forest (by default=10);
* max_features — the number of features to consider when looking for the best split;
* criterion — the function to measure the quality of a split ('mse' for regression, gini' or 'entropy' for classification);
* min_samples_leaf — the minimum number of samples required to be at a leaf node;
* max_depth — the maximum depth of the tree.

You could find more information about Randon Forest (and other ensembles) parameters [here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble).

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42, max_depth=20, max_features=0.7, class_weight='balanced')
rf.fit(X_train_new, y_train)
print(log_loss(y_val, rf.predict_proba(X_val_new)))

0.578809468263


### Random Forest Pros:
* it is one of the most accurate learning algorithms available. For many data sets, it produces a highly accurate classifier;
* runs efficiently on large databases;
* gives estimates of what variables are important in the classification;
* has an effective method for estimating missing data and maintains accuracy when a large proportion of the data are missing;
* has methods for balancing error in class population unbalanced data sets;

### Random Forest Cons:
* produces a lower accurary on 'sparse' data (e.g. text, bag of words, sparse matrix);
* can be overfitted for some datasets with noisy classification/regression tasks;
* difficult to interpret.

## Extremely Randomize Trees

or Extra Trees are very similar to Random Forest algorithm, but use differenrt approach to constructing the decion trees:

* each tree is built from the complete learning sample (doesn't apply the bagging procedure to construct a set of the training samples for each tree);
* for each of the features (randomly selected at each interior node) a discretization threshold (cut-point) is selected at random to define a split, instead of choosing the best cut-point based on the local sample (as in Tree Bagging or in the Random Forests methods).

In [21]:
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier(n_estimators=10, 
                             max_depth=20,
                             random_state=42,
                             n_jobs=-1,
                             class_weight='balanced',
                            )
extra.fit(X_train_new, y_train)
print(log_loss(y_val, extra.predict_proba(X_val_new)))

0.657443851713


The main advantage of Extratrees – preventing overfitting. Usually used on last stack layers.

## Gradient Boosting Algorithms

The common ensemble techniques like random forests rely on simple averaging of models in the ensemble. The family of boosting methods is based on a different, constructive strategy of ensemble formation. The main idea of boosting is to add new models to the ensemble sequentially. <br> 

In gradient boosting  the learning procedure consecutively fits new models to provide a more accurate estimate of the response variable. The principle idea behind this algorithm is to construct the new base-learners to be maximally correlated with the negative gradient of the loss function, associated with the whole ensemble. Simply put, the next algorithm tries to fix the error of the previous one. 

Briefly consider the constructing of gradient boosting algorithm:

* Step 1. Applying a base learning algorithm.
* Step 2. Initially all points have same weight (denoted by their size). After the first iteration the points classified correctly are given a lower weight and vice versa.
* Step 3. The next algorithm focuses on high weight points and try to classificate them correctly.
* Step 4. Iterate steps 2 and 3 till the limit of base learning algorithm is reached or higher accuracy is achieved or no longer improves on an external validation dataset.


We will consider XGBoost and LightGbm: gradient boosting frameworks that use tree based learning algorithms. 

### Boosting parameters
Tunning parameters is very important stage, we can signifincantly improve our baseline model. It is very difficult to get answers to practical questions like – Which set of parameters should be tuned ? Let's take a look on general GBm parameters

All GBM parameters can be divided in three main groups:
1. Tree-Specific Parameters;
2. Boosting Parameters;
3. Miscellaneous Parameters. 


The optimal parameters of a model can depend on many scenarios. So it is impossible to create a comprehensive guide for doing so. 

### XGBoost parameters tuning

Usually we start tuning parameters with these first: <br>
`n_estimators` - number of boosting rounds, better to use `early_stopping_rounds` <br>
`eta` – learning rate, increasing lr reduces convergence time. (usually default value works good)

### Control Overfitting
When you observe high training accuracy, but low tests accuracy, it is likely that you encounter overfitting problem.

There are in general two ways that you can control overfitting:

* The first way is to directly control model complexity
This include `max_depth` - maximum depth of a tree, increase this value will make the model more complex; <br>
`min_child_weight` - minimum sum of instance weight needed in a child; <br>
`gamma` - minimum loss reduction required to make a further partition on a leaf node of the tree.


* The second way is to add randomness to make training robust to noise
This include `subsample` - subsample ratio of the training instance,
`colsample_bytree` - subsample ratio of columns when constructing each tree. <br>


### Handle Imbalanced Dataset
There are two ways to improve it:

* If you care only about the ranking order (AUC) of your prediction
Balance the positive and negative weights, via `scale_pos_weight`
Use AUC for evaluation
* If you care about predicting the right probability
In such a case, you cannot re-balance the dataset
In such a case, set parameter `max_delta_step` to a finite number (say 1) will help convergence <br>

In [22]:
import xgboost as xgb

In [42]:
param = {}
param['objective'] = 'binary:logistic'
param['max_depth'] = 7
param['eta'] = .2
param['colsample_bytree'] = 0.7
param['subsample'] = 0.7
param['silent'] = 1
param['eval_metric'] = 'logloss'
num_round = 50

In [43]:
xgb_train = xgb.DMatrix(X_train_new, y_train)
xgb_val = xgb.DMatrix(X_val_new, y_val)



In [44]:
cv_results = xgb.cv(param, xgb_train,num_round, nfold=3, stratified=True, seed=42, verbose_eval=1, early_stopping_rounds=5)

[0]	train-logloss:0.669942+0.000848789	test-logloss:0.670174+0.000868563
[1]	train-logloss:0.652754+0.000587322	test-logloss:0.653121+0.000563452
[2]	train-logloss:0.6404+0.000945502	test-logloss:0.641028+0.00102469
[3]	train-logloss:0.631624+0.00116385	test-logloss:0.63242+0.00123753
[4]	train-logloss:0.624356+0.00141998	test-logloss:0.625482+0.00164696
[5]	train-logloss:0.617642+0.00059521	test-logloss:0.619071+0.000964797
[6]	train-logloss:0.612043+0.000718684	test-logloss:0.613626+0.000983719
[7]	train-logloss:0.607767+0.000983182	test-logloss:0.60957+0.001202
[8]	train-logloss:0.603497+0.00158781	test-logloss:0.605523+0.00193997
[9]	train-logloss:0.599887+0.00128126	test-logloss:0.602043+0.00165774
[10]	train-logloss:0.596506+0.00115907	test-logloss:0.598875+0.00155805
[11]	train-logloss:0.592819+0.000948224	test-logloss:0.595491+0.00126061
[12]	train-logloss:0.589821+0.000809305	test-logloss:0.592602+0.000764039
[13]	train-logloss:0.586856+0.00152947	test-logloss:0.589915+0.00119

In [53]:
evallist = [(xgb_train, "train"), (xgb_val, "val")]
model_xgboost = xgb.train(param, xgb_train, num_round, evallist, verbose_eval=1)

[0]	train-logloss:0.670007	val-logloss:0.670375
[1]	train-logloss:0.652499	val-logloss:0.653106
[2]	train-logloss:0.639016	val-logloss:0.639759
[3]	train-logloss:0.629368	val-logloss:0.630127
[4]	train-logloss:0.623003	val-logloss:0.624045
[5]	train-logloss:0.616863	val-logloss:0.617651
[6]	train-logloss:0.612027	val-logloss:0.613004
[7]	train-logloss:0.607925	val-logloss:0.60919
[8]	train-logloss:0.603408	val-logloss:0.604748
[9]	train-logloss:0.600034	val-logloss:0.6015
[10]	train-logloss:0.597286	val-logloss:0.598926
[11]	train-logloss:0.594394	val-logloss:0.596165
[12]	train-logloss:0.591621	val-logloss:0.593557
[13]	train-logloss:0.588962	val-logloss:0.591093
[14]	train-logloss:0.586079	val-logloss:0.58818
[15]	train-logloss:0.58283	val-logloss:0.58516
[16]	train-logloss:0.579157	val-logloss:0.58159
[17]	train-logloss:0.576871	val-logloss:0.579487
[18]	train-logloss:0.574985	val-logloss:0.577661
[19]	train-logloss:0.573319	val-logloss:0.576107
[20]	train-logloss:0.571735	val-loglo

### LightGbm parameters tuning

### For better accuracy

* Use large `num_leaves` - number of leaves in one tree (may cause over-fitting) <br>
* Use large `max_bin` (max number of bin that feature values will bucket in) <br>
* Use small `learning_rate` (shrinkage rate) with large `num_iterations` (only used in prediction task, used to how many trained iterations will be used in prediction) <br>
* Use bigger training data <br>


### For faster 

* Use bagging by set `bagging_fraction` (will random select part of data)  and `bagging_freq` (Frequency for bagging) <br>
* Use feature sub-sampling by set `feature_fraction` (will random select part of features on each iteration)
* Use small `max_bin`
* Use `save_binary` (will save the data set(include validation data) to a binary file)  to speed up data loading in future learning <br>
* Use parallel learning, refer to [parallel learning guide](https://github.com/Microsoft/LightGBM/wiki/Parallel-Learning-Guide) 

### Deal with over-fitting

* Use small `max_bin` <br>
* Use small `num_leaves` <br>
* Use `min_data_in_leaf` (minimal number of data in one leaf) <br>
* Use bagging by set `bagging_fraction` and `bagging_freq` <br>
* Use feature sub-sampling by set `feature_fraction` <br>
* Use bigger training data <br>
* Try `lambda_l1`, `lambda_l2` and `min_gain_to_split` to regularization <br>
* Try `max_depth` to avoid growing deep tree (tree still grows by leaf-wise) <br>

To learn more about tuning LightGbm hyperparameters please click [this link](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.md).

In [67]:
import lightgbm as lgb

params = {
    'application':'binary',
    'num_leaves': 31,
    'max_depth': 20,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'max_bin': 200,
    'metric': 'binary_logloss',
    'verbose': 1
    
}

lgb_train = lgb.Dataset(X_train_new.tocsc().astype("float32"), y_train)
lgb_val = lgb.Dataset(X_val_new.tocsc().astype("float32"), y_val)
results = lightgbm.cv(params, lgb_train, 100, nfold=3, stratified=True, early_stopping_rounds=5, verbose_eval=1)

[1]	cv_agg's binary_logloss: 0.678982 + 3.59888e-05
[2]	cv_agg's binary_logloss: 0.66741 + 0.000191059
[3]	cv_agg's binary_logloss: 0.656829 + 0.000315858
[4]	cv_agg's binary_logloss: 0.647587 + 0.000388539
[5]	cv_agg's binary_logloss: 0.639934 + 0.000720499
[6]	cv_agg's binary_logloss: 0.633132 + 0.000789629
[7]	cv_agg's binary_logloss: 0.627526 + 0.000862464
[8]	cv_agg's binary_logloss: 0.622271 + 0.000901618
[9]	cv_agg's binary_logloss: 0.617577 + 0.00101382
[10]	cv_agg's binary_logloss: 0.613441 + 0.00101246
[11]	cv_agg's binary_logloss: 0.609706 + 0.000898937
[12]	cv_agg's binary_logloss: 0.606256 + 0.000833069
[13]	cv_agg's binary_logloss: 0.603055 + 0.000580361
[14]	cv_agg's binary_logloss: 0.600184 + 0.000640182
[15]	cv_agg's binary_logloss: 0.597362 + 0.000580897
[16]	cv_agg's binary_logloss: 0.594715 + 0.000663676
[17]	cv_agg's binary_logloss: 0.592269 + 0.000627423
[18]	cv_agg's binary_logloss: 0.59008 + 0.000520252
[19]	cv_agg's binary_logloss: 0.587579 + 0.000453445
[20]	c

In [68]:
model_lgm = lightgbm.train(params, lgb_train, 100, valid_sets=lgb_val, verbose_eval=1)

[1]	valid_0's binary_logloss: 0.679
[2]	valid_0's binary_logloss: 0.667174
[3]	valid_0's binary_logloss: 0.656984
[4]	valid_0's binary_logloss: 0.647965
[5]	valid_0's binary_logloss: 0.640312
[6]	valid_0's binary_logloss: 0.633768
[7]	valid_0's binary_logloss: 0.627799
[8]	valid_0's binary_logloss: 0.622441
[9]	valid_0's binary_logloss: 0.61774
[10]	valid_0's binary_logloss: 0.613628
[11]	valid_0's binary_logloss: 0.609851
[12]	valid_0's binary_logloss: 0.606365
[13]	valid_0's binary_logloss: 0.603248
[14]	valid_0's binary_logloss: 0.600255
[15]	valid_0's binary_logloss: 0.59725
[16]	valid_0's binary_logloss: 0.593997
[17]	valid_0's binary_logloss: 0.591503
[18]	valid_0's binary_logloss: 0.589416
[19]	valid_0's binary_logloss: 0.587224
[20]	valid_0's binary_logloss: 0.585438
[21]	valid_0's binary_logloss: 0.583612
[22]	valid_0's binary_logloss: 0.581918
[23]	valid_0's binary_logloss: 0.58024
[24]	valid_0's binary_logloss: 0.578357
[25]	valid_0's binary_logloss: 0.576895
[26]	valid_0's 

## Tricks

### Simple average

In [70]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(n_jobs=-1)
lg.fit(X_train_new, y_train)
lg_preds = lg.predict_proba(X_val_new)
print(log_loss(y_val, lg_preds))

0.508149219898


In [73]:
lgbm_preds = model_lgm.predict(X_val_new.tocsc().astype("float32"))
print(log_loss(y_val, lgbm_preds))

Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x129765860>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/site-packages/xgboost-0.6-py3.5.egg/xgboost/core.py", line 669, in __del__
    _LIB.XGBoosterFree(self.handle)
AttributeError: 'Booster' object has no attribute 'handle'
Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x1229b9198>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/site-packages/xgboost-0.6-py3.5.egg/xgboost/core.py", line 669, in __del__
    _LIB.XGBoosterFree(self.handle)
AttributeError: 'Booster' object has no attribute 'handle'


0.521505699925


In [75]:
print(log_loss(y_val, (lgbm_preds + lg_preds[:,1]) / 2.))

0.490515057107


### Weighted average

In [81]:
print(log_loss(y_val, lgbm_preds*0.35 + lg_preds[:,1] * 0.65))

0.488606105114


In [86]:
lg = LogisticRegression(n_jobs=-1, C=0.1)
lg.fit(X_train_new, y_train)
lg_preds_2 = lg.predict_proba(X_val_new)
print(log_loss(y_val, lg_preds_2))

0.498213465692


In [89]:
print(log_loss(y_val, (lg_preds_2[:,1] + lg_preds[:,1]) / 2.))

0.499804060129


In [91]:
np.corrcoef([lg_preds[:,1], lg_preds_2[:,1], lgbm_preds])

array([[ 1.        ,  0.98043019,  0.75241795],
       [ 0.98043019,  1.        ,  0.81114385],
       [ 0.75241795,  0.81114385,  1.        ]])

### Cheating

In [116]:
test1 = pd.read_csv("/Users/vitaliyradchenko/Downloads/xgb_seed12357_n315.csv")
test2 = pd.read_csv("/Users/vitaliyradchenko/Downloads/xgb_seed12357_n480.csv")

In [117]:
test1["is_duplicate"] = test1["is_duplicate"]*0.55 + test2["is_duplicate"]*0.45

In [118]:
test1.to_csv("submit.csv", index=False)

### Blending

![img](https://alexanderdyakonov.files.wordpress.com/2017/03/stacking.png?w=1400)

### Stacking

![img](https://alexanderdyakonov.files.wordpress.com/2017/03/stacking-2b.png?w=1400)

Further reading: https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending/